In [1]:
from ltp import LTP

ltp = LTP()

seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
sdp = ltp.sdp(hidden, graph=False)
sdp

[[(1, 2, 'Agt'),
  (2, 0, 'Root'),
  (3, 2, 'Datv'),
  (4, 2, 'eEfft'),
  (5, 4, 'eEfft'),
  (6, 5, 'Pat'),
  (7, 2, 'mPunc')]]

In [2]:
seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
sdp = ltp.sdp(hidden, graph=True)
sdp

[[(1, 2, 'Agt'),
  (2, 0, 'Root'),
  (3, 2, 'Datv'),
  (3, 4, 'Agt'),
  (3, 5, 'Agt'),
  (4, 2, 'eEfft'),
  (5, 4, 'eEfft'),
  (6, 5, 'Pat'),
  (7, 2, 'mPunc')]]

In [11]:
# seg, hidden = ltp.seg(["我军缴获敌人四辆坦克"])
# seg, hidden = ltp.seg(["这房子朝南"])
sents=['打篮球的小姑娘', '他听到鞭炮声', '他写了本小说', '周杰伦的出生日期是哪天']
for s in sents:
    seg, hidden = ltp.seg([s])
    sdp = ltp.sdp(hidden, graph=False)
    print(s, sdp)

打篮球的小姑娘 [[(1, 4, 'rAgt'), (2, 1, 'Pat'), (3, 1, 'mDepd'), (4, 0, 'Root')]]
他听到鞭炮声 [[(1, 2, 'Agt'), (2, 0, 'Root'), (3, 2, 'Cont')]]
他写了本小说 [[(1, 2, 'Agt'), (2, 0, 'Root'), (3, 2, 'mDepd'), (4, 5, 'Meas'), (5, 2, 'Prod')]]
周杰伦的出生日期是哪天 [[(1, 4, 'Exp'), (2, 1, 'mDepd'), (3, 4, 'rTime'), (4, 5, 'Exp'), (5, 0, 'Root'), (6, 7, 'Sco'), (7, 5, 'Link')]]


In [15]:
seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
pos = ltp.pos(hidden)
seg, pos, ltp.ner(hidden)

([['他', '叫', '汤姆', '去', '拿', '外衣', '。']],
 [['r', 'v', 'nh', 'v', 'v', 'n', 'wp']],
 [[('Nh', 2, 2)]])

In [17]:
ners=ltp.ner(hidden)
for tag, start, end in ners[0]:
    print(tag,":", "".join(seg[0][start:end + 1]))

Nh : 汤姆


In [40]:
def print_args(sents):
    seg, hidden = ltp.seg([sents])
    srl=ltp.srl(hidden)
    for tok, term in zip(seg[0], srl[0]):
        print(tok, term)
        for tag, start, end in term:
            print('\t', tag, ":", "".join(seg[0][start:end + 1]))

print_args("叫汤姆在中午去拿外衣。")

叫 [('A1', 1, 1), ('A2', 2, 6)]
	 A1 : 汤姆
	 A2 : 在中午去拿外衣
汤姆 []
在 []
中午 []
去 []
拿 [('A0', 1, 1), ('ARGM-TMP', 2, 3), ('A1', 6, 6)]
	 A0 : 汤姆
	 ARGM-TMP : 在中午
	 A1 : 外衣
外衣 []
。 []


In [41]:
print_args("叫汤姆不要拿外衣。")

叫 [('A1', 1, 1), ('A2', 2, 4)]
	 A1 : 汤姆
	 A2 : 不要拿外衣
汤姆 []
不要 []
拿 [('A0', 1, 1), ('ARGM-ADV', 2, 2), ('A1', 4, 4)]
	 A0 : 汤姆
	 ARGM-ADV : 不要
	 A1 : 外衣
外衣 []
。 []


In [23]:
seg[0][1]

'叫'

In [29]:
def proc_sents(sents):
    seg, hidden = ltp.seg([sents])
    sdp = ltp.sdp(hidden, graph=False)
    for start, end, tag in sdp[0]:
        print(seg[0][start-1],f"({tag})", seg[0][end-1])

proc_sents("他叫汤姆去拿外衣。")

他 (Agt) 叫
叫 (Root) 。
汤姆 (Datv) 叫
去 (eEfft) 叫
拿 (eEfft) 去
外衣 (Pat) 拿
。 (mPunc) 叫


In [31]:
proc_sents("他打了小明。")

他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
小明 (Pat) 打
。 (mPunc) 打


In [34]:
proc_sents("小明被他打了。")
print('*'*15)
proc_sents("他打了小明一拳。")

小明 (Pat) 打
被 (mRela) 他
他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
。 (mPunc) 打
***************
他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
小明 (Pat) 打
一 (Meas) 拳
拳 (Meas) 打
。 (mPunc) 打


In [20]:
# ltp.srl(hidden, keep_empty=False)
ltp.srl(hidden)

[[[],
  [('A0', 0, 0), ('A1', 2, 2), ('A2', 3, 5)],
  [],
  [],
  [('A0', 2, 2), ('A1', 5, 5)],
  [],
  []]]